# Ollama Configuration and Setup

This notebook helps you configure and test your Ollama installation for use with LangChain.

**What this notebook covers:**
- Verify Ollama installation and connectivity
- Test model availability
- Configure connection settings
- Validate the setup for LangChain integration

In [ ]:
# Import required libraries
import sys
sys.path.append('..')  # Add parent directory to path

from config import config
import requests
import json

print(f"Using Ollama URL: {config.ollama_base_url}")
print(f"Default model: {config.default_model}")

In [ ]:
# Test Ollama connectivity
def check_ollama_status():
    """Check if Ollama server is running and accessible."""
    try:
        response = requests.get(f"{config.ollama_base_url}")
        if response.status_code == 200:
            print("✅ Ollama server is running and accessible!")
            return True
        else:
            print(f"❌ Ollama server responded with status: {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama server!")
        print("   Make sure Ollama is installed and running:")
        print("   - Install: https://ollama.ai/download")
        print("   - Start: ollama serve")
        return False
    except Exception as e:
        print(f"❌ Error checking Ollama status: {e}")
        return False

ollama_running = check_ollama_status()

In [ ]:
# Get list of available models
models = []
if ollama_running:
    print("🔍 Checking available models...")
    
    try:
        response = requests.get(f"{config.ollama_base_url}/api/tags")
        if response.status_code == 200:
            data = response.json()
            models = [model['name'] for model in data.get('models', [])]
        else:
            print(f"Error getting model list: {response.status_code}")
    except Exception as e:
        print(f"Error getting models: {e}")

print(f"Found {len(models)} installed models:")
if models:
    for model in models:
        print(f"  - {model}")
else:
    print("  No models found. You may need to pull some models first.")
    print("  Example: ollama pull llama2")

In [ ]:
# Test default model
if config.default_model in models:
    print(f"✅ Default model '{config.default_model}' is available!")
elif models:
    print(f"⚠️  Default model '{config.default_model}' not found.")
    print("   Available models:")
    for model in models:
        print(f"     - {model}")
    print(f"   Consider updating config.py or pulling the model:")
    print(f"   ollama pull {config.default_model}")
else:
    print(f"❌ No models available. Please pull some models first:")
    print(f"   ollama pull {config.default_model}")

In [ ]:
# Test basic model interaction (if available)
if models and config.default_model in models:
    print(f"🧪 Testing basic interaction with {config.default_model}...")
    
    try:
        test_payload = {
            "model": config.default_model,
            "prompt": "Hello! Please respond with just 'Test successful'",
            "stream": False
        }
        
        response = requests.post(
            f"{config.ollama_base_url}/api/generate",
            json=test_payload,
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Model test successful!")
            print(f"   Response: {result.get('response', 'No response')[:100]}...")
        else:
            print(f"❌ Model test failed with status: {response.status_code}")
            
    except Exception as e:
        print(f"❌ Model test failed: {e}")
else:
    print("⚠️  Skipping model test (no compatible models available)")

## Summary

If all tests above passed, you're ready to use Ollama with LangChain!

**Next steps:**
1. If tests failed, check the Ollama installation and make sure it's running
2. Pull required models: `ollama pull llama2` (or your preferred model)  
3. Update `config.py` if needed
4. Proceed to the example notebooks